<a href="https://colab.research.google.com/github/asia281/dnn2022/blob/main/Asia_Bootcamp_ML%2C_Lab_4_logistic_regression_student_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Let's start with the simplest example - linear separated points on a plane.

In [1]:
import numpy as np

np.random.seed(123)

# these parametrize the line
a = 0.3
b = -0.2
c = 0.001

# True/False mapping
def lin_rule(x, noise=0.):
    return a * x[0] + b * x[1] + c + noise < 0.

# Just for plotting
def get_y_fun(a, b, c):
    def y(x):
        return - x * a / b - c / b
    return y

lin_fun = get_y_fun(a, b, c)

In [2]:
# Training data

n = 500
range_points = 1
sigma = 0.05

X = range_points * 2 * (np.random.rand(n, 2) - 0.5)
y = [lin_rule(x, sigma * np.random.normal()) for x in X]

print(X[:10])
print(y[:10])

[[ 0.39293837 -0.42772133]
 [-0.54629709  0.10262954]
 [ 0.43893794 -0.15378708]
 [ 0.9615284   0.36965948]
 [-0.0381362  -0.21576496]
 [-0.31364397  0.45809941]
 [-0.12285551 -0.88064421]
 [-0.20391149  0.47599081]
 [-0.63501654 -0.64909649]
 [ 0.06310275  0.06365517]]
[False, True, False, False, False, True, False, True, True, False]


Let's plot the data.

In [3]:
import plotly.express as px

# plotly has a problem with coloring boolean values, hence stringify
# see https://community.plotly.com/t/plotly-express-scatter-color-not-showing/25962
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))
x_range = [np.min(X[:, 0]), np.max(X[:, 1])]
fig.add_scatter(x=x_range, y=list(map(lin_fun, x_range)), name='ground truth border')
fig.show()

Now, let's implement and train a logistic regression model. You should obtain accuracy of at least 96%.

# Logistic regression

In this exercise you will train a logistic regression model via gradient descent in two simple scenarios.

The general setup is as follows:
* we are given a set of pairs $(x, y)$, where $x \in R^D$ is a vector of real numbers representing the features, and $y \in \{0,1\}$ is the target,
* for a given $x$ we model the probability of $y=1$ by $h(x):=g(w^Tx)$, where $g$ is the sigmoid function: $g(z) = \frac{1}{1+e^{-z}}$,
* to find the right $w$ we will optimize the so called logarithmic loss: $J(w) = -\frac{1}{n}\sum_{i=1}^n y_i \log{h(x_i)} + (1-y_i) \log{(1-h(x_i))}$,
* with the loss function in hand we can improve our guesses iteratively:
    * $w_j^{t+1} = w_j^t - \text{step_size} \cdot \frac{\partial J(w)}{\partial w_j}$,
* we can end the process after some predefined number of epochs (or when the changes are no longer meaningful).

In [4]:
################################################################
# TODO: Implement logistic regression and compute its accuracy #
################################################################

class logistic_regression:
  def __init__(self, x, y):
    self._x = np.c_[x, np.ones(x.shape[0])]
    self._y = y

  def sigmoid(self, z):
    return 1 / (1 + np.exp(-z))

  def predict(self, w, x):
    return self.sigmoid(np.dot(w, x.T))

  def logarithmic_loss(self, w, x, y):
    h = self.predict(w, x)
    mean = 0
    for idx, y_i in enumerate(y):
      mean += y_i * np.log(h[idx]) + (1 - y_i) * np.log(1 - h[idx])
    return -mean / h.shape[0]

  def deriv(self, w):
    dot = -np.dot(self._x.T, self._y - self.predict(w, self._x))
    return dot / self._x.shape[0]

  def fit(self, lr, epochs, w):
    if w is None:
      w = np.zeros(self._x.shape[1])
    for i in range(epochs):
      w -= lr * self.deriv(w).T
      print(f'{i}: {self.logarithmic_loss(w, self._x, self._y)}')
    return w


In [5]:
epochs = 1000
lr = 0.5
logistic_reg = logistic_regression(X, y)
w = logistic_reg.fit(lr, epochs, None)
w = logistic_reg.fit(lr/10, epochs*10, w)

Streaming output truncated to the last 5000 lines.
5000: 0.10613591882117013
5001: 0.10613482491826355
5002: 0.10613373114271409
5003: 0.1061326374944994
5004: 0.1061315439735977
5005: 0.1061304505799868
5006: 0.10612935731364494
5007: 0.10612826417454996
5008: 0.10612717116268015
5009: 0.10612607827801322
5010: 0.10612498552052727
5011: 0.10612389289020034
5012: 0.1061228003870108
5013: 0.10612170801093626
5014: 0.10612061576195489
5015: 0.10611952364004493
5016: 0.10611843164518407
5017: 0.10611733977735086
5018: 0.10611624803652299
5019: 0.1061151564226787
5020: 0.10611406493579618
5021: 0.10611297357585327
5022: 0.10611188234282821
5023: 0.1061107912366991
5024: 0.10610970025744389
5025: 0.10610860940504095
5026: 0.10610751867946824
5027: 0.106106428080704
5028: 0.1061053376087263
5029: 0.10610424726351296
5030: 0.1061031570450426
5031: 0.10610206695329316
5032: 0.10610097698824268
5033: 0.10609988714986945
5034: 0.10609879743815157
5035: 0.10609770785306726
5036: 0.106096618394594

Let's visually asses our model. We can do this by using our estimates for $a,b,c$.

In [6]:
#################################################################
# TODO: Pass your estimates for a,b,c to the get_y_fun function #
#################################################################
lin_fun2 = get_y_fun(w[0], w[1], w[2])

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))
x_range = [np.min(X[:, 0]), np.max(X[:, 1])]
fig.add_scatter(x=x_range, y=list(map(lin_fun, x_range)), name='ground truth border')
fig.add_scatter(x=x_range, y=list(map(lin_fun2, x_range)), name='estimated border')
fig.show()

Let's now complicate the things a little bit and make our next problem nonlinear.

In [7]:
# Parameters of the ellipse
s1 = 1.
s2 = 2.
r = 0.75
m1 = 0.15
m2 = 0.125

# 0/1 mapping, checks whether we are inside the ellipse
def circle_rule(x, y, noise=0.):
    return 1 if s1 * (x - m1) ** 2 + s2 * (y - m2) ** 2 + noise < r ** 2 else 0

In [8]:
# Training data

n = 500
range_points = 1

sigma = 0.1

X = range_points * 2 * (np.random.rand(n, 2) - 0.5)

y = [circle_rule(x, y, sigma * np.random.normal()) for x, y in X]

print(X[:10])
print(y[:10])

[[ 0.18633789  0.87560968]
 [-0.81999293  0.61838609]
 [ 0.22604784  0.28001611]
 [ 0.9846182  -0.35783437]
 [-0.27962406  0.07170775]
 [ 0.2501677  -0.37650776]
 [ 0.41264707 -0.8357508 ]
 [-0.61039043 -0.97349628]
 [ 0.49924022  0.89579621]
 [ 0.537422   -0.65425777]]
[0, 0, 1, 0, 1, 1, 0, 0, 0, 0]


Let's plot the data.

In [9]:
import plotly.graph_objects as go

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))

xgrid = np.arange(np.min(X[:, 0]), np.max(X[:, 0]), 0.003)
ygrid = np.arange(np.min(X[:, 1]), np.max(X[:, 1]), 0.003)
contour =  go.Contour(
        z=np.vectorize(circle_rule)(*np.meshgrid(xgrid, ygrid, indexing="ij")),
        x=xgrid,
        y=ygrid
    )
fig.add_trace(contour)
fig.show()

Now, let's train a logistic regression model to tackle this problem. Note that we now need a nonlinear decision boundary. You should obtain accuracy of at least 90%.

Hint:
<sub><sup><sub><sup><sub><sup>
Use feature engineering.
</sup></sub></sup></sub></sup></sub>

In [10]:
################################################################
# TODO: Implement logistic regression and compute its accuracy #
################################################################
xs = np.array(X)
ys = np.array(y)
def scale(xs):
  return np.c_[xs, xs[:,0]**2, xs[:,1]**2, xs[:, 0] * xs[:, 1]]


def predict_sec(xs, ws):
  xs = np.c_[np.array(xs), np.ones(xs.shape[0])]
  return np.sigmoid(np.dot(xs, ws))

new_xs = scale(xs)
log_reg = logistic_regression(new_xs, ys)
W = log_reg.fit(0.5, 100000, None)

Streaming output truncated to the last 5000 lines.
95000: 0.08609650252044157
95001: 0.08609649621600748
95002: 0.08609648991183252
95003: 0.08609648360791704
95004: 0.08609647730426064
95005: 0.08609647100086343
95006: 0.08609646469772551
95007: 0.08609645839484667
95008: 0.08609645209222713
95009: 0.08609644578986676
95010: 0.08609643948776544
95011: 0.08609643318592337
95012: 0.08609642688434033
95013: 0.08609642058301667
95014: 0.08609641428195187
95015: 0.08609640798114622
95016: 0.08609640168059973
95017: 0.08609639538031226
95018: 0.08609638908028372
95019: 0.08609638278051453
95020: 0.08609637648100414
95021: 0.08609637018175297
95022: 0.08609636388276072
95023: 0.08609635758402762
95024: 0.08609635128555339
95025: 0.086096344987338
95026: 0.08609633868938174
95027: 0.08609633239168456
95028: 0.08609632609424626
95029: 0.08609631979706689
95030: 0.08609631350014639
95031: 0.08609630720348468
95032: 0.08609630090708201
95033: 0.08609629461093829
95034: 0.08609628831505343
95035:

Let's visually asses our model.

Contrary to the previous scenario, converting our weights to parameters of the ground truth curve may not be straightforward. It's easier to just provide predictions for a set of points in $R^2$.

In [11]:
h = .02

xgrid = np.arange(np.min(X[:, 0]), np.max(X[:, 0]), h)
ygrid = np.arange(np.min(X[:, 1]), np.max(X[:, 1]), h)

xx, yy = np.meshgrid(xgrid, ygrid, indexing="ij")
X_plot = np.c_[xx.ravel(), yy.ravel()]

print(X_plot.shape)

_X = np.concatenate([X_plot, X_plot**2], axis=1)

preds = logistic_regression(_w, _b, _X)
print(preds.shape)


(10000, 2)


NameError: ignored

In [ ]:
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))

xx, yy = np.meshgrid(xgrid, ygrid, indexing="ij")

contour = go.Contour(z=preds.reshape(len(xgrid), len(ygrid)), x=xgrid, y=ygrid)
fig.add_trace(contour)
fig.show()

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>